In [ ]:
# -------------------------------
# 1. Setup & Imports
# -------------------------------

import os
import cv2
import torch
import mmcv
from mmdet.apis import init_detector, inference_detector, single_gpu_test
from mmdet.datasets import build_dataset, build_dataloader
from mmdet.models import build_detector
from mmdet.apis import set_random_seed
from mmdet.apis import train_detector
from mmcv.runner import load_checkpoint
import matplotlib.pyplot as plt

# -------------------------------
# 2. Config & Model Initialization
# -------------------------------
# Path to Swin Transformer config (BDD100K detection config)
config_file = '/home/badri/bosch-project/part2_model/swin_config.py'

# Path to pretrained checkpoint downloaded from model zoo
checkpoint_file = '/home/badri/bosch-project/part2_model/swin_cascade.pth'

# Initialize model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = init_detector(config_file, checkpoint_file, device=device)

# -------------------------------
# 3. Build Validation Dataset
# -------------------------------
from mmdet.datasets import build_dataset, build_dataloader

cfg = mmcv.Config.fromfile(config_file)
cfg.data.test.ann_file = '/home/badri/bosch-project/data/labels/bdd100k_labels_images_val.json'
cfg.data.test.img_prefix = '/home/badri/bosch-project/data/images/val'
dataset = build_dataset(cfg.data.test)

data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=2,
    dist=False,
    shuffle=False
)

# -------------------------------
# 4. Run Inference & Evaluation
# -------------------------------
outputs = single_gpu_test(model, data_loader, show=False)

# Evaluate with standard metrics
results = dataset.evaluate(outputs, metric=['bbox'])
print("Evaluation Results:", results)

# -------------------------------
# 5. Visualization
# -------------------------------
# Show predictions on a few val images
sample_images = [
    '/home/badri/bosch-project/data/images/val/b1c9c847-3bda4659.jpg'
]

for img_path in sample_images:
    result = inference_detector(model, img_path)
    model.show_result(img_path, result, out_file=None, score_thr=0.4)

# -------------------------------
# 6. Class-wise AP Visualization
# -------------------------------
import numpy as np

if 'bbox_mAP_50' in results:
    class_names = dataset.CLASSES
    ap_per_class = results['bbox_mAP_c']  # per-class AP
    plt.figure(figsize=(12,6))
    plt.bar(range(len(class_names)), ap_per_class)
    plt.xticks(range(len(class_names)), class_names, rotation=45)
    plt.ylabel("AP")
    plt.title("Per-class Average Precision on BDD100K val")
    plt.show()


ModuleNotFoundError: No module named 'mmcv._ext'